## Thuật toán sử dụng công thức Gauss 2 để giải đa thức nội suy 

In [2]:
import numpy as np
import pandas as pd
import math
from numpy.polynomial import polynomial as P

def load_and_validate_gauss_data(file_x_path, file_y_path):
    """Hàm đọc dữ liệu từ file và kiểm tra các điều kiện cho nội suy Gauss."""
    try:
        x_coords = np.loadtxt(file_x_path, ndmin=1).astype(float)
        y_coords = np.loadtxt(file_y_path, ndmin=1).astype(float)
    except (FileNotFoundError, ValueError):
        raise IOError("LỖI: Không tìm thấy file hoặc file chứa dữ liệu không hợp lệ.")

    if len(x_coords) != len(y_coords):
        raise ValueError("LỖI: Số lượng điểm x và y không khớp.")

    if len(np.unique(x_coords)) != len(x_coords):
        raise ValueError("LỖI: Các giá trị x không được trùng lặp.")
        
    # Kiểm tra điều kiện quan trọng: các mốc x phải cách đều
    diffs = np.diff(x_coords)
    if not np.allclose(diffs, diffs[0]):
        raise ValueError("LỖI: Các mốc nội suy x phải cách đều nhau để áp dụng công thức Gauss.")
        
    return x_coords, y_coords

# --- Chương trình chính cho Ô 1 ---
try:
    x_coords, y_coords = load_and_validate_gauss_data('FileX.txt', 'FileY.txt')
    print("Dữ liệu hợp lệ và đã được đọc thành công!")
    print("-" * 50)
    print("Các mốc nội suy x:", x_coords.tolist())
    print("Các giá trị y tương ứng:", y_coords.tolist())
except (IOError, ValueError) as e:
    print(str(e))


Dữ liệu hợp lệ và đã được đọc thành công!
--------------------------------------------------
Các mốc nội suy x: [1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6]
Các giá trị y tương ứng: [0.524, -0.084, -0.964, -2.121, -3.558, -5.272, -7.255, -9.494, -11.973]


In [3]:
# Giả định x_coords và y_coords đã tồn tại từ Ô 1

def create_advanced_difference_table(x_values, y_values):
    """Tạo bảng sai phân đầy đủ, đã được làm gọn."""
    num_points = len(y_values)
    center_idx = num_points // 2
    
    diff_table_data = [[0.0 for _ in range(num_points)] for _ in range(num_points)]
    for i in range(num_points):
        diff_table_data[i][0] = y_values[i]
    for j in range(1, num_points):
        for i in range(num_points - j):
            diff_table_data[i][j] = diff_table_data[i+1][j-1] - diff_table_data[i][j-1]
            
    headers = [f'Δ^{i}y' for i in range(1, num_points)]
    df = pd.DataFrame(np.array(diff_table_data)[:, 1:], columns=headers)
    df.insert(0, 'y', y_values)
    df.insert(0, 'x', x_values)
    positions = [f'x_{i - center_idx}' for i in range(num_points)]
    df.insert(0, 'Vị trí', positions)
            
    return df, np.array(diff_table_data)

# Tạo lại bảng và lấy dữ liệu để tính toán
df_advanced, diff_table_for_calc = create_advanced_difference_table(x_coords, y_coords)

# --- Bắt đầu phần định dạng và tô màu ---
center_idx = len(x_coords) // 2
gauss2_path_coords = set()
gauss2_path_coords.add((center_idx, 'y'))
for j in range(1, len(x_coords)):
    row_idx = center_idx - ((j + 1) // 2)
    col_name = f'Δ^{j}y'
    gauss2_path_coords.add((row_idx, col_name))

def highlight_gauss2_path(data):
    attr = 'background-color: yellow; color: black; font-weight: bold;'
    default = ''
    df_style = pd.DataFrame(default, index=data.index, columns=data.columns)
    for row, col in gauss2_path_coords:
        if col in df_style.columns and row in df_style.index:
            df_style.loc[row, col] = attr
    return df_style

print("--- Bảng sai phân nâng cao (Tô màu theo đường đi Gauss II) ---")

numeric_cols = df_advanced.select_dtypes(include=np.number).columns

# >>> THAY ĐỔI Ở ĐÂY: Sử dụng định dạng "{:g}" để không làm tròn <<<
styled_df = df_advanced.style.apply(highlight_gauss2_path, axis=None).format(
    "{:g}", subset=numeric_cols, na_rep=""
)

display(styled_df)

--- Bảng sai phân nâng cao (Tô màu theo đường đi Gauss II) ---


,Vị trí,x,y,Δ^1y,Δ^2y,Δ^3y,Δ^4y,Δ^5y,Δ^6y,Δ^7y,Δ^8y
0,x_-4,1,0.524,-0.608,-0.272,-0.005,0.002,0.004,-0.005,0.006,-0.009
1,x_-3,1.2,-0.084,-0.88,-0.277,-0.003,0.006,-0.001,0.001,-0.003,0
2,x_-2,1.4,-0.964,-1.157,-0.28,0.003,0.005,-4.44089e-15,-0.002,0,0
3,x_-1,1.6,-2.121,-1.437,-0.277,0.008,0.005,-0.002,0,0,0
4,x_0,1.8,-3.558,-1.714,-0.269,0.013,0.003,0,0,0,0
5,x_1,2,-5.272,-1.983,-0.256,0.016,0,0,0,0,0
6,x_2,2.2,-7.255,-2.239,-0.24,0,0,0,0,0,0
7,x_3,2.4,-9.494,-2.479,0,0,0,0,0,0,0
8,x_4,2.6,-11.973,0,0,0,0,0,0,0,0


In [8]:
# Giả định x_coords và y_coords đã tồn tại từ Ô 1

def create_advanced_difference_table(x_values, y_values):
    """Tạo bảng sai phân đầy đủ, đã được làm gọn."""
    num_points = len(y_values); center_idx = num_points // 2
    diff_table_data = [[0.0 for _ in range(num_points)] for _ in range(num_points)]
    for i in range(num_points): diff_table_data[i][0] = y_values[i]
    for j in range(1, num_points):
        for i in range(num_points - j): diff_table_data[i][j] = diff_table_data[i+1][j-1] - diff_table_data[i][j-1]
    headers = [f'Δ^{i}y' for i in range(1, num_points)]; df = pd.DataFrame(np.array(diff_table_data)[:, 1:], columns=headers)
    df.insert(0, 'y', y_values); df.insert(0, 'x', x_values)
    positions = [f'x_{i - center_idx}' for i in range(num_points)]; df.insert(0, 'Vị trí', positions)
    return df, np.array(diff_table_data)

# Tạo lại bảng và lấy dữ liệu để tính toán
df_advanced, diff_table_for_calc = create_advanced_difference_table(x_coords, y_coords)

# --- Phần định dạng và tô màu ---
center_idx = len(x_coords) // 2
gauss2_path_coords = set()
gauss2_path_coords.add((center_idx, 'y'))
for j in range(1, len(x_coords)):
    row_idx = center_idx - ((j + 1) // 2)
    col_name = f'Δ^{j}y'
    gauss2_path_coords.add((row_idx, col_name))

def highlight_gauss2_path(data):
    attr = 'background-color: yellow; color: black; font-weight: bold;'
    default = ''
    df_style = pd.DataFrame(default, index=data.index, columns=data.columns)
    for row, col in gauss2_path_coords:
        if col in df_style.columns and row in df_style.index:
            df_style.loc[row, col] = attr
    return df_style

print("--- Bảng sai phân nâng cao (Tô màu theo đường đi Gauss II, không làm tròn) ---")

# >>> THAY ĐỔI Ở ĐÂY: Bỏ hoàn toàn phần .format() <<<
# Chúng ta vẫn áp dụng tô màu, nhưng không áp dụng định dạng số nữa.
styled_df = df_advanced.style.apply(highlight_gauss2_path, axis=None)

display(styled_df)

--- Bảng sai phân nâng cao (Tô màu theo đường đi Gauss II, không làm tròn) ---


,Vị trí,x,y,Δ^1y,Δ^2y,Δ^3y,Δ^4y,Δ^5y,Δ^6y,Δ^7y,Δ^8y
0,x_-4,1.000000,0.524000,-0.608000,-0.272000,-0.005000,0.002000,0.004000,-0.005000,0.006000,-0.009000
1,x_-3,1.200000,-0.084000,-0.880000,-0.277000,-0.003000,0.006000,-0.001000,0.001000,-0.003000,0.000000
2,x_-2,1.400000,-0.964000,-1.157000,-0.280000,0.003000,0.005000,-0.000000,-0.002000,0.000000,0.000000
3,x_-1,1.600000,-2.121000,-1.437000,-0.277000,0.008000,0.005000,-0.002000,0.000000,0.000000,0.000000
4,x_0,1.800000,-3.558000,-1.714000,-0.269000,0.013000,0.003000,0.000000,0.000000,0.000000,0.000000
5,x_1,2.000000,-5.272000,-1.983000,-0.256000,0.016000,0.000000,0.000000,0.000000,0.000000,0.000000
6,x_2,2.200000,-7.255000,-2.239000,-0.240000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,x_3,2.400000,-9.494000,-2.479000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,x_4,2.600000,-11.973000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
def create_advanced_difference_table(x_values, y_values):
    """Tạo bảng sai phân đầy đủ, đã được làm gọn."""
    num_points = len(y_values)
    center_idx = num_points // 2
    
    # Tạo bảng sai phân cho y (ma trận trên)
    # diff_table_data[i][j] sẽ chứa Δ^j y_i
    diff_table_data = [[0.0 for _ in range(num_points)] for _ in range(num_points)]
    for i in range(num_points):
        diff_table_data[i][0] = y_values[i]
    for j in range(1, num_points):
        for i in range(num_points - j):
            diff_table_data[i][j] = diff_table_data[i+1][j-1] - diff_table_data[i][j-1]
            
    # >>> SỬA Ở ĐÂY: Tạo tiêu đề cột không bị trùng lặp <<<
    # Bắt đầu từ Δ^1y, vì cột y đã có riêng
    headers = [f'Δ^{i}y' for i in range(1, num_points)]
    
    # Tạo DataFrame chỉ từ các cột sai phân
    df = pd.DataFrame(np.array(diff_table_data)[:, 1:], columns=headers)
    
    # Thêm các cột x và y vào vị trí đầu
    df.insert(0, 'y', y_values)
    df.insert(0, 'x', x_values)
    
    # Thêm cột "Vị trí"
    positions = [f'x_{i - center_idx}' for i in range(num_points)]
    df.insert(0, 'Vị trí', positions)
            
    return df, np.array(diff_table_data)

# Tạo và hiển thị lại bảng
df_advanced, diff_table_for_calc = create_advanced_difference_table(x_coords, y_coords)

print("--- Bảng sai phân nâng cao ---")
# Áp dụng định dạng chỉ cho các cột số
numeric_cols = df_advanced.select_dtypes(include=[np.number]).columns
display(df_advanced)

--- Bảng sai phân nâng cao ---


,Vị trí,x,y,Δ^1y,Δ^2y,Δ^3y,Δ^4y,Δ^5y,Δ^6y,Δ^7y,Δ^8y
0,x_-4,1.0,0.524,-0.608,-0.272,-0.005,0.002,4.000000e-03,-0.005,0.006,-0.009
1,x_-3,1.2,-0.084,-0.880,-0.277,-0.003,0.006,-1.000000e-03,0.001,-0.003,0.000
2,x_-2,1.4,-0.964,-1.157,-0.280,0.003,0.005,-4.440892e-15,-0.002,0.000,0.000
3,x_-1,1.6,-2.121,-1.437,-0.277,0.008,0.005,-2.000000e-03,0.000,0.000,0.000
4,x_0,1.8,-3.558,-1.714,-0.269,0.013,0.003,0.000000e+00,0.000,0.000,0.000
5,x_1,2.0,-5.272,-1.983,-0.256,0.016,0.000,0.000000e+00,0.000,0.000,0.000
6,x_2,2.2,-7.255,-2.239,-0.240,0.000,0.000,0.000000e+00,0.000,0.000,0.000
7,x_3,2.4,-9.494,-2.479,0.000,0.000,0.000,0.000000e+00,0.000,0.000,0.000
8,x_4,2.6,-11.973,0.000,0.000,0.000,0.000,0.000000e+00,0.000,0.000,0.000


In [5]:
# Giả định các biến x_coords và diff_table_for_calc đã có từ các ô trước

def get_gauss_2_coeffs_in_t(x_data, full_diff_table):
    """
    Tính các hệ số của đa thức P(t) trực tiếp từ công thức Gauss II.
    """
    n = len(x_data) - 1
    center_idx = n // 2
    final_poly_in_t = np.array([full_diff_table[center_idx][0]])
    product_poly_in_t = np.array([1.0])
    
    for i in range(1, n + 1):
        if i == 1:
            factor_poly = np.array([0.0, 1.0])
        elif i % 2 == 0:
            k = i / 2
            factor_poly = np.array([k, 1.0])
        else:
            k = (i - 1) / 2
            factor_poly = np.array([-k, 1.0])
            
        product_poly_in_t = P.polymul(product_poly_in_t, factor_poly)
        diff_row_idx = center_idx - ((i + 1) // 2)
        diff_val = full_diff_table[diff_row_idx][i]
        term_poly_in_t = (diff_val / math.factorial(i)) * product_poly_in_t
        final_poly_in_t = P.polyadd(final_poly_in_t, term_poly_in_t)
        
    return final_poly_in_t

# >>> THÊM VÀO ĐÂY: Hàm định dạng đa thức bị thiếu <<<
def format_polynomial_descending(coeffs_asc):
    """Hàm định dạng đa thức, hiển thị tất cả các số hạng không bằng 0."""
    coeffs_desc = coeffs_asc[::-1]
    poly_str = ""
    n = len(coeffs_desc) - 1
    
    for i, coeff in enumerate(coeffs_desc):
        if coeff == 0.0:
            continue
            
        power = n - i
        sign = " - " if coeff < 0 else " + "
        coeff_abs = abs(coeff)
        
        if poly_str == "":
            sign = "" if coeff > 0 else "-"
            
        if np.isclose(coeff_abs, 1) and power != 0:
            coeff_str = ""
        else:
            coeff_str = f"{coeff_abs:g}"
            
        if power == 1:
            var_str = "x"
        elif power == 0:
            var_str = ""
        else:
            var_str = f"x^{power}"
            
        separator = "*" if coeff_str != "" and var_str != "" else ""
        poly_str += f" {sign} {coeff_str}{separator}{var_str}"
        
    return poly_str.strip()


# --- Thực thi và hiển thị ---
coeffs_in_t_gauss2 = get_gauss_2_coeffs_in_t(x_coords, diff_table_for_calc)

print("--- Các hệ số của đa thức theo ẩn t, P(t) [Gauss II] ---")
print(coeffs_in_t_gauss2)

print("\n--- 'Hình dạng' của đa thức P(t) (từ Gauss II) ---")
# Giờ dòng này sẽ chạy được vì hàm đã được định nghĩa ở trên
print(f"P(t) =", format_polynomial_descending(coeffs_in_t_gauss2).replace('x', 't'))

print("\n(Lưu ý: Kết quả P(t) phải giống với kết quả tính bằng Gauss I)")

--- Các hệ số của đa thức theo ẩn t, P(t) [Gauss II] ---
[-3.55800000e+00 -1.57644405e+00 -1.38694742e-01  9.52083333e-04
  1.90451389e-04 -8.33333333e-06  4.51388889e-06  2.97619048e-07
 -2.23214286e-07]

--- 'Hình dạng' của đa thức P(t) (từ Gauss II) ---
P(t) = - 2.23214e-07*t^8  +  2.97619e-07*t^7  +  4.51389e-06*t^6  -  8.33333e-06*t^5  +  0.000190451*t^4  +  0.000952083*t^3  -  0.138695*t^2  -  1.57644*t  -  3.558

(Lưu ý: Kết quả P(t) phải giống với kết quả tính bằng Gauss I)


In [6]:
# Giả định các biến x_coords và diff_table_for_calc đã có từ các ô trước

def get_gauss_1_coeffs_in_t(x_data, full_diff_table):
    """
    Tính các hệ số của đa thức P(t) trực tiếp từ công thức Gauss I.
    """
    n = len(x_data) - 1
    center_idx = n // 2
    
    # Bắt đầu với số hạng đầu tiên của công thức: y_0
    final_poly_in_t = np.array([full_diff_table[center_idx][0]])
    
    # Đa thức tích (basis polynomial), ban đầu là 1
    product_poly_in_t = np.array([1.0])
    
    # Lặp qua từng số hạng tiếp theo trong công thức
    for i in range(1, n + 1):
        
        # Xây dựng đa thức tích P_i(t)
        if i == 1:
            factor_poly = np.array([0.0, 1.0])  # t
        elif i % 2 == 0:
            k = i / 2
            factor_poly = np.array([-k, 1.0])  # (t - k)
        else:
            k = (i - 1) / 2
            factor_poly = np.array([k, 1.0])   # (t + k)
        product_poly_in_t = P.polymul(product_poly_in_t, factor_poly)
        
        # Lấy giá trị sai phân tương ứng
        diff_row_idx = center_idx - (i // 2)
        diff_val = full_diff_table[diff_row_idx][i]
        
        # Tạo đa thức cho toàn bộ số hạng
        term_poly_in_t = (diff_val / math.factorial(i)) * product_poly_in_t
        
        # Cộng đa thức của số hạng này vào kết quả cuối cùng
        final_poly_in_t = P.polyadd(final_poly_in_t, term_poly_in_t)
        
    return final_poly_in_t

# --- Thực thi và định nghĩa biến coeffs_in_t ---
coeffs_in_t = get_gauss_1_coeffs_in_t(x_coords, diff_table_for_calc)

# In kết quả để xác nhận
print("--- Biến 'coeffs_in_t' đã được định nghĩa ---")
print(coeffs_in_t)

--- Biến 'coeffs_in_t' đã được định nghĩa ---
[-3.55800000e+00 -1.57644405e+00 -1.38694742e-01  9.52083333e-04
  1.90451389e-04 -8.33333333e-06  4.51388889e-06  2.97619048e-07
 -2.23214286e-07]


In [7]:
import math
import numpy as np
import pandas as pd

# >>> THAY ĐỔI: Thiết lập độ chính xác hiển thị cho Pandas <<<
# Yêu cầu pandas hiển thị các số thực với 15 chữ số sau dấu phẩy
pd.set_option('display.precision', 15)


# Giả định các biến sau đã tồn tại từ các ô code bạn đã chạy trước:
# x_coords: Danh sách các mốc nội suy x ban đầu.
# coeffs_in_t: Các hệ số của đa thức P(t) theo bậc tăng dần.

# --- 1. Chuẩn bị dữ liệu và chuyển đổi x -> t ---

# Đa thức P(t) có hệ số theo bậc giảm dần để dùng cho Horner
p_coeffs_descending = coeffs_in_t[::-1] 

# Giá trị X bạn muốn tính đạo hàm
x_input = 9.68 

# Tính toán các giá trị x0, h để chuyển đổi
n = len(x_coords) - 1
center_idx = n // 2
x0 = x_coords[center_idx]
h = x_coords[1] - x_coords[0]

# Áp dụng công thức chuyển đổi x -> t
t_val = (x_input - x0) / h

print("--- Bước 1: Chuyển đổi X sang T ---")
print(f"Giá trị x đầu vào: x = {x_input}")
print(f"Mốc trung tâm: x0 = {x0}, bước nhảy h = {h}")
print(f"Giá trị t tương ứng: t = ({x_input} - {x0}) / {h} = {t_val}")


# --- 2. Hàm tính đạo hàm bằng Horner (đã bỏ làm tròn) ---

def tinh_bang_dao_ham_horner(coeffs_desc, t_value_to_eval):
    """
    Thực hiện Horner lặp để tính tất cả đạo hàm từ P(t) đến P^(n)(t).
    """
    print("\n" + "="*50)
    print(f"Đa thức P(t) có hệ số (bậc giảm dần): {coeffs_desc}")
    print(f"Tính các đạo hàm tại t = {t_value_to_eval}")
    
    current_coeffs = list(coeffs_desc)
    n = len(current_coeffs) - 1
    summary_data = []
    k_factorial = 1.0

    # Lặp qua từng cấp đạo hàm, từ k = 0 đến n
    for k in range(n + 1):
        print("\n" + "="*50)
        print(f"--- BƯỚC {k+2}.{k+1}: TÍNH TOÁN CHO ĐẠO HÀM CẤP {k} ---")
        
        m = len(current_coeffs)
        deg = m - 1
        
        # --- Hiển thị bảng Horner cho bước hiện tại ---
        header = [f'a_{deg-i}' for i in range(deg + 1)]
        row_coeffs = current_coeffs
        row_products = ['']
        row_results = []

        b = current_coeffs[0]
        row_results.append(b)
        
        quotient_coeffs = [b]
        for i in range(1, m):
            product = b * t_value_to_eval
            row_products.append(product)
            b = product + current_coeffs[i]
            row_results.append(b)
            if i < m -1:
                quotient_coeffs.append(b)
        
        remainder = b
        
        df = pd.DataFrame(
            [row_coeffs, row_products, row_results],
            index=['Hệ số vào', f'Nhân với t={t_value_to_eval}', 'Kết quả'],
            columns=header
        )
        display(df)
        
        # --- Tính giá trị đạo hàm ---
        if k > 0:
            k_factorial *= k
        
        derivative_value = remainder * k_factorial
        summary_data.append([f"P^({k})(t)", derivative_value])
        
        print(f"-> Số dư R_{k+1} = {remainder}")
        print(f"-> Giá trị đạo hàm P^({k})(t) = R_{k+1} * {k}! = {remainder} * {k_factorial} = {derivative_value}")

        current_coeffs = quotient_coeffs

    # --- In bảng tổng kết cuối cùng ---
    summary_df = pd.DataFrame(summary_data, columns=["Đạo hàm", f"Giá trị tại t = {t_value_to_eval}"])
    print("\n" + "="*50)
    print(f"--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = {x_input} ---")
    display(summary_df)

# --- 3. Thực thi hàm ---
tinh_bang_dao_ham_horner(p_coeffs_descending, t_val)

--- Bước 1: Chuyển đổi X sang T ---
Giá trị x đầu vào: x = 9.68
Mốc trung tâm: x0 = 1.8, bước nhảy h = 0.19999999999999996
Giá trị t tương ứng: t = (9.68 - 1.8) / 0.19999999999999996 = 39.400000000000006

Đa thức P(t) có hệ số (bậc giảm dần): [-2.23214286e-07  2.97619048e-07  4.51388889e-06 -8.33333333e-06
  1.90451389e-04  9.52083333e-04 -1.38694742e-01 -1.57644405e+00
 -3.55800000e+00]
Tính các đạo hàm tại t = 39.400000000000006

--- BƯỚC 2.1: TÍNH TOÁN CHO ĐẠO HÀM CẤP 0 ---


,a_8,a_7,a_6,a_5,a_4,a_3,a_2,a_1,a_0
Hệ số vào,-0.000000223214286,0.000000297619048,0.000004513888889,-0.000008333333333,0.000190451388889,0.000952083333333,-0.138694742063492,-1.576444047619048,-3.558000000000000e+00
Nhân với t=39.400000000000006,,-0.000008794642857,-0.000334782738094,-0.013012592658691,-0.513024484085745,-20.205660888256126,-796.065526913958138,-31370.446333247258735,-1.236057697425418e+06
Kết quả,-0.000000223214286,-0.000008497023809,-0.000330268849205,-0.013020925992024,-0.512834032696856,-20.204708804922792,-796.204221656021673,-31372.022777294878324,-1.236061255425418e+06


-> Số dư R_1 = -1236061.2554254183
-> Giá trị đạo hàm P^(0)(t) = R_1 * 0! = -1236061.2554254183 * 1.0 = -1236061.2554254183

--- BƯỚC 3.2: TÍNH TOÁN CHO ĐẠO HÀM CẤP 1 ---


,a_7,a_6,a_5,a_4,a_3,a_2,a_1,a_0
Hệ số vào,-0.000000223214286,-0.000008497023809,-0.000330268849205,-0.013020925992024,-0.512834032696856,-20.204708804922792,-796.204221656021673,-31372.022777294878324
Nhân với t=39.400000000000006,,-0.000008794642857,-0.000681291666665,-0.039855484325277,-2.083330566501663,-102.288885208421675,-4826.247604125772341,-221524.601935802726075
Kết quả,-0.000000223214286,-0.000017291666667,-0.001011560515870,-0.052876410317301,-2.596164599198519,-122.493594013344463,-5622.451825781794469,-252896.624713097611675


-> Số dư R_2 = -252896.6247130976
-> Giá trị đạo hàm P^(1)(t) = R_2 * 1! = -252896.6247130976 * 1.0 = -252896.6247130976

--- BƯỚC 4.3: TÍNH TOÁN CHO ĐẠO HÀM CẤP 2 ---


...

,a_1,a_0
Hệ số vào,-0.000000223214286,-0.000061264880952
Nhân với t=39.400000000000006,,-0.000008794642857
Kết quả,-0.000000223214286,-0.000070059523809


-> Số dư R_8 = -7.005952380931793e-05
-> Giá trị đạo hàm P^(7)(t) = R_8 * 7! = -7.005952380931793e-05 * 5040.0 = -0.35309999999896236

--- BƯỚC 10.9: TÍNH TOÁN CHO ĐẠO HÀM CẤP 8 ---


,a_0
Hệ số vào,-0.000000223214286
Nhân với t=39.400000000000006,
Kết quả,-0.000000223214286


-> Số dư R_9 = -2.2321428571363334e-07
-> Giá trị đạo hàm P^(8)(t) = R_9 * 8! = -2.2321428571363334e-07 * 40320.0 = -0.008999999999973696

--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = 9.68 ---


,Đạo hàm,Giá trị tại t = 39.400000000000006
0,P^(0)(t),-1.236061255425418e+06
1,P^(1)(t),-2.528966247130976e+05
2,P^(2)(t),-4.525491387815644e+04
3,P^(3)(t),-6.937869317047208e+03
4,P^(4)(t),-8.859045867640239e+02
5,P^(5)(t),-9.045315599973145e+01
6,P^(6)(t),-6.923269999979547e+00
7,P^(7)(t),-3.530999999989624e-01
8,P^(8)(t),-8.999999999973696e-03
